In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import sklearn
import nltk
import spacy
import matplotlib.pyplot as plt

# NLTK library preparation

In [2]:
# nltk.download()

In [3]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('polish')

# Spacy library preparation
#### You have to download the model for the Polish language
#### Be carefull where you install it, it should be strictly where your python is working
#### if tou operate on conda environment, you should install it in the same environment

In [2]:
# !python -m spacy download pl_core_news_sm
nlp = spacy.load("pl_core_news_sm")

# Read the data

#### Path to root directory

In [58]:
# get the root directory
root_dir = os.path.abspath(os.path.join(os.getcwd()))
print(root_dir)

/home/lizardman/Uczelnia/DataMining/Advanced-Data-Mining


#### Read data from pickle to dataframe

In [4]:
# i want to read tematy_i_opisy_df_cleaneed.pickle
path_to_pickle = os.path.join(root_dir, 'data_cleaning-data', 'tematy_i_opisy_df_cleaned.pickle')
df_main: pd.DataFrame = pd.read_pickle(path_to_pickle)

In [5]:
df_main.columns

Index(['Tytuł pracy w języku oryginału', 'Temat Pracy',
       'Jednostka organizacyjna', 'Osoba zgłaszająca temat',
       'Kierunki studiów', 'Typ pracy', 'Status tematu', 'Data aktualizacji',
       'url'],
      dtype='object')

In [6]:
df_main.shape

(5311, 9)

#### Additional cleaning

In [7]:
# get rows from df_main where 'Temat Pracy' is not null or len > 6
df_main_cleaned = df_main[df_main['Temat Pracy'].notnull()]
df_main_cleaned = df_main[df_main_cleaned['Temat Pracy'].str.len() > 1]
df_main_cleaned

,Tytuł pracy w języku oryginału,Temat Pracy,Jednostka organizacyjna,Osoba zgłaszająca temat,Kierunki studiów,Typ pracy,Status tematu,Data aktualizacji,url
0,Rozwój funkcjonalności oraz modernizacja konst...,Tematem niniejszej pracy jest rozwój funkcjona...,[110-000] Wydział Inżynierii Metali i Informat...,Piotr Kustra,Informatyka Techniczna,magisterska,Zarezerwowany,9 października 2024,https://apd.usos.agh.edu.pl/topics/show/15260/
1,Opracowanie rusztowania na bazie wełny owczej ...,Niniejsza praca dyplomowa skupia się na opraco...,"[120-000] Wydział Elektrotechniki, Automatyki,...",Katarzyna Nowicka-Dunal,inżynieria biomedyczna,inżynierska,Dostępny,8 października 2024,https://apd.usos.agh.edu.pl/topics/show/15244/
2,Papierowe systemy analityczne z mikroprzepływe...,Praca dotyczy prostych systemów diagnostycznyc...,[160-000] Wydział Inżynierii Materiałowej i Ce...,Bogusław Baś,Technologia Chemiczna,inżynierska,Zarezerwowany,8 października 2024,https://apd.usos.agh.edu.pl/topics/show/15243/
3,System wizyzjny odczytujący osobom niewidomym ...,"Celem pracy jest stworzenie systemu wizyjnego,...","[120-000] Wydział Elektrotechniki, Automatyki,...",Piotr Pawlik,Automatyka i Robotyka,inżynierska,Dostępny,8 października 2024,https://apd.usos.agh.edu.pl/topics/show/15241/
4,System wizyzjny nakierowujący osoby niewidome ...,"Celem pracy jest stworzenie systemu wizyjnego,...","[120-000] Wydział Elektrotechniki, Automatyki,...",Piotr Pawlik,Automatyka i Robotyka,inżynierska,Dostępny,8 października 2024,https://apd.usos.agh.edu.pl/topics/show/15240/
...,...,...,...,...,...,...,...,...,...
5306,Biznesplan sklepu z odzieżą używaną,W pracy zostaną przedstawione kolejne etapy po...,[200-000] Wydział Zarządzania,Małgorzata Białas,Zarządzanie,licencjacka,Zarezerwowany,17 maja 2021,https://apd.usos.agh.edu.pl/topics/show/172/
5307,Ewolucja bankowości elektronicznej w Polsce na...,W pracy dokonano opisu rozwoju bankowości elek...,[200-000] Wydział Zarządzania,Małgorzata Białas,Zarządzanie,licencjacka,Zarezerwowany,16 maja 2021,https://apd.usos.agh.edu.pl/topics/show/170/
5308,Ocena wykonania budżetu jednostki samorządu te...,W pracy zostanie przedstawiony planowany budże...,[200-000] Wydział Zarządzania,Małgorzata Białas,Zarządzanie,licencjacka,Dostępny,5 maja 2021,https://apd.usos.agh.edu.pl/topics/show/128/
5309,Rozwiązania smart-city stosowane dla krakowski...,W pracy przedstawione zostanie zainteresowanie...,[150-000] Wydział Geodezji Górniczej i Inżynie...,Elżbieta Jasińska,Geodezja i Kartografia,inżynierska,Dostępny,15 kwietnia 2021,https://apd.usos.agh.edu.pl/topics/show/46/


#### Extract the description and the title

In [8]:
descriptions = pd.DataFrame()
descriptions['Temat Pracy'] = df_main_cleaned['Tytuł pracy w języku oryginału']
descriptions['Opis Pracy'] = df_main_cleaned['Temat Pracy']

In [9]:
descriptions['Opis Pracy'][0]

'Tematem niniejszej pracy jest rozwój funkcjonalności oraz konstrukcji maszyny realizującej proces beznarzędziowego ciągnienia.\nPrzebudowanie systemu transportu układu grzewczego umożliwi zminimalizowanie zakłóceń procesu beznarzędziowego ciągnienia takich jak drgania systemu, które wynikają z zastosowania niskobudżetowego napędu liniowego. Efektem tych działań będzie otrzymanie próbek oraz danych z procesu beznarzędziowego ciągnienia o wyższej jakości.\nAdaptacja kodu aplikacji do pracy na komputerze stacjonarnym zwiększy częstotliwości odczytu wartości z czujnika siły, wykonywania zdjęć próbki oraz podniesie wydajność algorytmu analizującego obraz i wyliczającego wartość odkształcenia próbki. Wynikiem zwiększenia wydajności tych operacji jest uzyskanie dokładniejszych wykresów naprężenia od odkształcenia oraz siły od wydłużenia próbki. Ponadto umożliwi to wykonywanie statycznych prób rozciągania przy większych prędkościach odkształcenia próbki.\nZamontowanie enkodera absolutnego na 

In [10]:
descriptions.head(1)

,Temat Pracy,Opis Pracy
0,Rozwój funkcjonalności oraz modernizacja konst...,Tematem niniejszej pracy jest rozwój funkcjona...


In [27]:
polish_to_english = str.maketrans({
    'ą': 'a', 'ć': 'c', 'ę': 'e', 'ł': 'l', 'ń': 'n', 
    'ó': 'o', 'ś': 's', 'ź': 'z', 'ż': 'z'
})

descriptions['Cleaned Tokens'] = descriptions['Opis Pracy'].apply(lambda x: [token.text.translate(polish_to_english) for token in nlp(x)])
descriptions.head()

,Temat Pracy,Opis Pracy,Cleaned Tokens
0,Rozwój funkcjonalności oraz modernizacja konst...,Tematem niniejszej pracy jest rozwój funkcjona...,"[Tematem, niniejszej, pracy, jest, rozwoj, fun..."
1,Opracowanie rusztowania na bazie wełny owczej ...,Niniejsza praca dyplomowa skupia się na opraco...,"[Niniejsza, praca, dyplomowa, skupia, sie, na,..."
2,Papierowe systemy analityczne z mikroprzepływe...,Praca dotyczy prostych systemów diagnostycznyc...,"[Praca, dotyczy, prostych, systemow, diagnosty..."
3,System wizyzjny odczytujący osobom niewidomym ...,"Celem pracy jest stworzenie systemu wizyjnego,...","[Celem, pracy, jest, stworzenie, systemu, wizy..."
4,System wizyzjny nakierowujący osoby niewidome ...,"Celem pracy jest stworzenie systemu wizyjnego,...","[Celem, pracy, jest, stworzenie, systemu, wizy..."


In [29]:
descriptions['Cleaned Tokens'][0]

['Tematem',
 'niniejszej',
 'pracy',
 'jest',
 'rozwoj',
 'funkcjonalnosci',
 'oraz',
 'konstrukcji',
 'maszyny',
 'realizujacej',
 'proces',
 'beznarzedziowego',
 'ciagnienia',
 '.',
 '\n',
 'Przebudowanie',
 'systemu',
 'transportu',
 'ukladu',
 'grzewczego',
 'umozliwi',
 'zminimalizowanie',
 'zaklocen',
 'procesu',
 'beznarzedziowego',
 'ciagnienia',
 'takich',
 'jak',
 'drgania',
 'systemu',
 ',',
 'ktore',
 'wynikaja',
 'z',
 'zastosowania',
 'niskobudzetowego',
 'napedu',
 'liniowego',
 '.',
 'Efektem',
 'tych',
 'dzialan',
 'bedzie',
 'otrzymanie',
 'probek',
 'oraz',
 'danych',
 'z',
 'procesu',
 'beznarzedziowego',
 'ciagnienia',
 'o',
 'wyzszej',
 'jakosci',
 '.',
 '\n',
 'Adaptacja',
 'kodu',
 'aplikacji',
 'do',
 'pracy',
 'na',
 'komputerze',
 'stacjonarnym',
 'zwiekszy',
 'czestotliwosci',
 'odczytu',
 'wartosci',
 'z',
 'czujnika',
 'sily',
 ',',
 'wykonywania',
 'zdjec',
 'probki',
 'oraz',
 'podniesie',
 'wydajnosc',
 'algorytmu',
 'analizujacego',
 'obraz',
 'i',
 'w

In [13]:
descriptions['Cleaned Tokens'][0]

['Tematem',
 'niniejszej',
 'pracy',
 'jest',
 'rozwój',
 'funkcjonalności',
 'oraz',
 'konstrukcji',
 'maszyny',
 'realizującej',
 'proces',
 'beznarzędziowego',
 'ciągnienia',
 '.',
 '\n',
 'Przebudowanie',
 'systemu',
 'transportu',
 'układu',
 'grzewczego',
 'umożliwi',
 'zminimalizowanie',
 'zakłóceń',
 'procesu',
 'beznarzędziowego',
 'ciągnienia',
 'takich',
 'jak',
 'drgania',
 'systemu',
 ',',
 'które',
 'wynikają',
 'z',
 'zastosowania',
 'niskobudżetowego',
 'napędu',
 'liniowego',
 '.',
 'Efektem',
 'tych',
 'działań',
 'będzie',
 'otrzymanie',
 'próbek',
 'oraz',
 'danych',
 'z',
 'procesu',
 'beznarzędziowego',
 'ciągnienia',
 'o',
 'wyższej',
 'jakości',
 '.',
 '\n',
 'Adaptacja',
 'kodu',
 'aplikacji',
 'do',
 'pracy',
 'na',
 'komputerze',
 'stacjonarnym',
 'zwiększy',
 'częstotliwości',
 'odczytu',
 'wartości',
 'z',
 'czujnika',
 'siły',
 ',',
 'wykonywania',
 'zdjęć',
 'próbki',
 'oraz',
 'podniesie',
 'wydajność',
 'algorytmu',
 'analizującego',
 'obraz',
 'i',
 'w

In [14]:
path_to_data_dir = "/home/lizardman/Uczelnia/DataMining/Advanced-Data-Mining/feature_generation-data"
path_to_data = os.path.join(path_to_data_dir, 'description.pickle')

print(os.path.isdir(path_to_data_dir))

True


In [30]:
with open(path_to_data, 'wb') as f:
    pickle.dump(descriptions, f)

#### Add a column with the length of the description and a column with tokens from the description

In [32]:
descriptions1 = pd.read_pickle(path_to_data)

#### Tokens to lower case

In [33]:
descriptions1['Cleaned Tokens'] = (descriptions1["Cleaned Tokens"]
                                   .apply(lambda row: [token.lower() for token in row]))
descriptions1['Cleaned Tokens'][0]

['tematem',
 'niniejszej',
 'pracy',
 'jest',
 'rozwoj',
 'funkcjonalnosci',
 'oraz',
 'konstrukcji',
 'maszyny',
 'realizujacej',
 'proces',
 'beznarzedziowego',
 'ciagnienia',
 '.',
 '\n',
 'przebudowanie',
 'systemu',
 'transportu',
 'ukladu',
 'grzewczego',
 'umozliwi',
 'zminimalizowanie',
 'zaklocen',
 'procesu',
 'beznarzedziowego',
 'ciagnienia',
 'takich',
 'jak',
 'drgania',
 'systemu',
 ',',
 'ktore',
 'wynikaja',
 'z',
 'zastosowania',
 'niskobudzetowego',
 'napedu',
 'liniowego',
 '.',
 'efektem',
 'tych',
 'dzialan',
 'bedzie',
 'otrzymanie',
 'probek',
 'oraz',
 'danych',
 'z',
 'procesu',
 'beznarzedziowego',
 'ciagnienia',
 'o',
 'wyzszej',
 'jakosci',
 '.',
 '\n',
 'adaptacja',
 'kodu',
 'aplikacji',
 'do',
 'pracy',
 'na',
 'komputerze',
 'stacjonarnym',
 'zwiekszy',
 'czestotliwosci',
 'odczytu',
 'wartosci',
 'z',
 'czujnika',
 'sily',
 ',',
 'wykonywania',
 'zdjec',
 'probki',
 'oraz',
 'podniesie',
 'wydajnosc',
 'algorytmu',
 'analizujacego',
 'obraz',
 'i',
 'w

#### Remove punctuation

In [35]:
# remove punctuation
import string

descriptions1['Cleaned Tokens'] = descriptions1['Cleaned Tokens'].apply(
    lambda row: [word for word in row if word not in string.punctuation])

descriptions1["Cleaned Tokens"][1]

['niniejsza',
 'praca',
 'dyplomowa',
 'skupia',
 'sie',
 'na',
 'opracowaniu',
 'innowacyjnego',
 'rusztowania',
 'do',
 'regeneracji',
 'tkanek',
 'bazujacego',
 'na',
 'naturalnym',
 'biomateriale',
 'jakim',
 'jest',
 'welna',
 'owcza',
 'glownym',
 'celem',
 'pracy',
 'jest',
 'opracowanie',
 'i',
 'charakterystyka',
 'rusztowania',
 'na',
 'bazie',
 'welny',
 'owczej',
 'modyfikowany',
 'materialami',
 'hydrozelowymi',
 'ktore',
 'spelnialoby',
 'kryteria',
 'pozwalajace',
 'zastosowac',
 'otrzymany',
 'kompozyt',
 'jako',
 'material',
 'opatrunkowy',
 'material',
 'taki',
 'musi',
 'charakteryzowac',
 'sie',
 'odpowiedna',
 'porowatoscia',
 'wytrzymaloscia',
 'mechaniczna',
 'nasiakliwoscia',
 'i',
 'biokompatybilnoscia',
 'w',
 'ramach',
 'pracy',
 'przygotowane',
 'zostana',
 'rusztowania',
 'na',
 'bazie',
 'welny',
 'owczej',
 'z',
 'oczyszczonych',
 'wlokien',
 'modyfikowanych',
 'materialem',
 'hydrozelowym',
 'charakterystyka',
 'rusztowan',
 'bedzie',
 'obejmowala',
 'ob

TypeError: sequence item 0: expected str instance, list found

In [57]:
descriptions1["Cleaned Tokens"]=descriptions1["Cleaned Tokens"].apply(lambda row: [token for token in row if token not in nlp.Defaults.stop_words])
descriptions1["Cleaned Tokens"][0]

['tematem',
 'niniejszej',
 'pracy',
 'rozwoj',
 'funkcjonalnosci',
 'konstrukcji',
 'maszyny',
 'realizujacej',
 'proces',
 'beznarzedziowego',
 'ciagnienia',
 '\n',
 'przebudowanie',
 'systemu',
 'transportu',
 'ukladu',
 'grzewczego',
 'umozliwi',
 'zminimalizowanie',
 'zaklocen',
 'procesu',
 'beznarzedziowego',
 'ciagnienia',
 'drgania',
 'systemu',
 'wynikaja',
 'zastosowania',
 'niskobudzetowego',
 'napedu',
 'liniowego',
 'efektem',
 'dzialan',
 'otrzymanie',
 'probek',
 'danych',
 'procesu',
 'beznarzedziowego',
 'ciagnienia',
 'wyzszej',
 'jakosci',
 '\n',
 'adaptacja',
 'kodu',
 'aplikacji',
 'pracy',
 'komputerze',
 'stacjonarnym',
 'zwiekszy',
 'czestotliwosci',
 'odczytu',
 'wartosci',
 'czujnika',
 'sily',
 'wykonywania',
 'zdjec',
 'probki',
 'podniesie',
 'wydajnosc',
 'algorytmu',
 'analizujacego',
 'obraz',
 'wyliczajacego',
 'wartosc',
 'odksztalcenia',
 'probki',
 'wynikiem',
 'zwiekszenia',
 'wydajnosci',
 'operacji',
 'uzyskanie',
 'dokladniejszych',
 'wykresow',

In [25]:
# Define the Polish to English character mapping


# descriptions1['Cleaned Tokens'] = descriptions1['Cleaned Tokens'].apply(
#     lambda row: [token.text.translate(polish_to_english) for token in row])

token0: list[str] = descriptions1['Cleaned Tokens'][0]
token0

"['tematem', 'niniejszej', 'pracy', 'jest', 'rozwój', 'funkcjonalności', 'oraz', 'konstrukcji', 'maszyny', 'realizującej', 'proces', 'beznarzędziowego', 'ciągnienia', '\\n', 'przebudowanie', 'systemu', 'transportu', 'układu', 'grzewczego', 'umożliwi', 'zminimalizowanie', 'zakłóceń', 'procesu', 'beznarzędziowego', 'ciągnienia', 'takich', 'jak', 'drgania', 'systemu', 'które', 'wynikają', 'z', 'zastosowania', 'niskobudżetowego', 'napędu', 'liniowego', 'efektem', 'tych', 'działań', 'będzie', 'otrzymanie', 'próbek', 'oraz', 'danych', 'z', 'procesu', 'beznarzędziowego', 'ciągnienia', 'o', 'wyższej', 'jakości', '\\n', 'adaptacja', 'kodu', 'aplikacji', 'do', 'pracy', 'na', 'komputerze', 'stacjonarnym', 'zwiększy', 'częstotliwości', 'odczytu', 'wartości', 'z', 'czujnika', 'siły', 'wykonywania', 'zdjęć', 'próbki', 'oraz', 'podniesie', 'wydajność', 'algorytmu', 'analizującego', 'obraz', 'i', 'wyliczającego', 'wartość', 'odkształcenia', 'próbki', 'wynikiem', 'zwiększenia', 'wydajności', 'tych', 'o